<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./2_2_S6_advanced_data_access_hda.ipynb">Acquiring a full Sentinel-6 cycle >></a>

<font color="#138D75">**Copernicus Marine Training Service**</font> <br>
**Copyright:** 2022 EUMETSAT <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h3>Learn Sentinel-6: Advanced</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
Users should review the following notebooks for more information on setting up a credentials files for using the eumdac python library to retrieve Poseidon-4 data from the Data Store:
    
- **<a href="../1_S6_introductory/1_1b_S6_data_access_DS.ipynb">1_1b_S6_data_access_DS.ipynb</a>**

</div>
<hr>

# 2.1 Poseidon-4 advanced data access via the Data Store using eumdac

### Data used

| Product Description  | Data Store collection ID| Product Navigator | 
|:--------------------:|:-----------------------:|:-------------:|
| Sentinel-6 Poseidon-4 Altimetry level-2 Low Resolution - Reprocessed | EO:EUM:DAT:0239 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:0239?query=0239&s=advanced " target="_blank">link</a> |

### Learning outcomes

At the end of this notebook you will know;
* How to refine your <font color="#138D75">**searches**</font> for Sentinel-6 products in the EUMETSAT Data Store using the `eumdac` API client
* How to <font color="#138D75">**download**</font> components of products

### Outline

The EUMETSAT Data Store offers many ways to interact with data in order to refine searches. Many of these methods are supported by the EUMETSAT Data Access Client (`eumdac`). In this notebook we will showcase some of the possibilities for using `eumdac` to better interact with Sentinel-6 collections. This notebook assumes that you already have an understanding of the available Data Store interfaces, which you can gain by running the **<a href="../1_S6_introductory/1_1b_S6_data_access_DS.ipynb">1_1b_S6_data_access_DS.ipynb</a>** notebook.

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
1. [Example 1: Filter by collections](#section1)
1. [Example 2: Filter by time](#section2)
1. [Example 3: Filter by space and time](#section3)
1. [Example 4: Download by component](#section4)

<hr>

We begin by importing all of the libraries that we need to run this notebook. If you have built your python using the environment file provided in this repository, then you should have everything you need. For more information on building environment, please see the repository **<a href="../README.md" target="_blank">README</a>**.

In [1]:
import os
import json
import datetime
import shutil
import eumdac
import inspect
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import eumartools
from shapely import geometry, vectorized

# Create a download directory for our Sentinel-6 products
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

Before we use the Data Store to download data, we must first authenticate our access and retrieve an access token. More information on setting this up can be found in the **<a href="../1_S6_introductory/1_1b_S6_data_access_DS.ipynb">1_1b_S6_data_access_DS.ipynb</a>** notebook.

In [2]:
# load credentials
with open(os.path.join(os.path.expanduser("~"),'.eumdac_credentials')) as json_file:
    credentials = json.load(json_file)
    token = eumdac.AccessToken((credentials['consumer_key'], credentials['consumer_secret']))
    print(f"This token '{token}' expires {token.expiration}")

# create data store object
datastore = eumdac.DataStore(token)



This token 'f35822b2-9368-387a-85a5-05013574439d' expires 2022-09-27 20:09:08.829957


We will work with the Sentinel-6 level-2 low resolution data throughout this notebook. The collection ID for this data type is `EO:EUM:DAT:0239`.

In [3]:
# set collection ID for Sentinel-6 level-2 LR Reprocessed
collectionID = 'EO:EUM:DAT:0239'

<div class="alert alert-info" role="alert">

## <a id='section1'></a>Example 1: Filtering by collection
[Back to top](#TOC_TOP)

</div>

To filter by collection, we simply provide the collectionID to the `datastore.get_collection method`

In [4]:
# Use collection ID
selected_collection = datastore.get_collection(collectionID)
print(selected_collection.title)

Poseidon-4 Altimetry Level 2 Low Resolution (baseline version F06) - Sentinel-6 - Reprocessed


<div class="alert alert-info" role="alert">

## <a id='section2'></a>Example 2: Filtering by time
[Back to top](#TOC_TOP)

</div>

To get the latest product in a collection, we can just ask for the `.first()` method, when searching in our selected collection using `.search()`.

In [5]:
# Get the latest product in a collection
selected_collection = datastore.get_collection(collectionID)
latest = selected_collection.search().first()
print(latest)

S6A_P4_2__LR______20220428T003339_20220428T012952_20220522T193943_3373_054_022_011_EUM__REP_NT_F06.SEN6


To filter by time, we can pass python datetime arguments to the **dtstart** and **dtend** arguments of our collection when using the `.search()` method.

In [6]:
# time filter the collection for products
start = datetime.datetime(2022, 1, 23, 16, 0)
end = datetime.datetime(2022, 1, 23, 16, 15)
products = selected_collection.search(dtstart=start, dtend=end)

for product in products:
    print(product)

S6A_P4_2__LR______20220123T160250_20220123T165903_20220427T034113_3373_044_145_072_EUM__REP_NT_F06.SEN6
S6A_P4_2__LR______20220123T150637_20220123T160250_20220427T033742_3373_044_144_072_EUM__REP_NT_F06.SEN6


<div class="alert alert-info" role="alert">

## <a id='section3'></a>Example 3: Filtering by space and time
[Back to top](#TOC_TOP)

</div>

We can also add geographical filtering by passing in a <a href="https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry" target="_blank">Well Known Text</a> (WKT) format region of interest into the **geo** argument of the `.search()` method.

In [7]:
# space/time filter the collection for products
selected_collection = datastore.get_collection(collectionID)
start = datetime.datetime(2022, 1, 23)
end = datetime.datetime(2022, 1, 24)
roi = [[-1.0, -1.0], [4.0, -4.0], [8.0, -2.0], [9.0, 2.0], [6.0, 4.0], [1.0, 5.0], [-1.0, -1.0]]

products = selected_collection.search(
    geo='POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in roi])),
    dtstart=start, 
    dtend=end)

for product in products:
    print(product)

S6A_P4_2__LR______20220123T064041_20220123T073654_20220427T024643_3373_044_135_067_EUM__REP_NT_F06.SEN6


<div class="alert alert-info" role="alert">

## <a id='section4'></a>Example 4: Downloading by component
[Back to top](#TOC_TOP)

</div>

It is possible to download any component of a SAFE format file. This can be achieved by using iterating through the `entries` attribute of a product and string matching to your component of interest - in this case the manifest file. 

*Note: It is not currently possible to download a single variable from **inside** a netCDF file*.

In [8]:
# Get the latest product in a collection
selected_collection = datastore.get_collection(collectionID)
latest = selected_collection.search().first()

for entry in latest.entries:
    if 'xfdumanifest.xml' in entry:
        with latest.open(entry=entry) as fsrc, open(os.path.join(os.getcwd(), 'products',fsrc.name),
                                                    mode='wb') as fdst:
            print(f'Downloading {fsrc.name}.')
            shutil.copyfileobj(fsrc, fdst)
            print(f'Download of file {fsrc.name} finished.')

Download of file xfdumanifest.xml finished.


<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./2_2_S6_advanced_data_access_hda.ipynb">Acquiring a full Poseidon-4 cycle >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>